# Convolutional Neural Networks (CNNs)

## Quelle der Daten

https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database?resource=download (zuletzt aufgerufen 01/2024)
    
M.E.H. Chowdhury, T. Rahman, A. Khandakar, R. Mazhar, M.A. Kadir, Z.B. Mahbub, K.R. Islam, M.S. Khan, A. Iqbal, N. Al-Emadi, M.B.I. Reaz, M. T. Islam, “Can AI help in screening Viral and COVID-19 pneumonia?” IEEE Access, Vol. 8, 2020, pp. 132665 - 132676.

Rahman, T., Khandakar, A., Qiblawey, Y., Tahir, A., Kiranyaz, S., Kashem, S.B.A., Islam, M.T., Maadeed, S.A., Zughaier, S.M., Khan, M.S. and Chowdhury, M.E., 2020. Exploring the Effect of Image Enhancement Techniques on COVID-19 Detection using Chest X-ray Images.

## Installation der Bibliotheken

In [ ]:
# Bibliotheken importieren, die für das Laden, Vorverarbeiten und Modellieren der Daten benötigt werden.
import os

import cv2

import matplotlib.pyplot as plt

import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import requests

import tempfile

import shutil

## Download der Daten

In [ ]:
# URL des Datensatzes festlegen und temporäres Verzeichnis für den Download erstellen.
# load dataset from online resource, given here: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database
ds_url = "https://drive.usercontent.google.com/download?id=1bum9Sehb3AzUMHLhBMuowPKyr_PCrB3a&export=download&confirm=1"
temp_dir = tempfile.mkdtemp()
print(f"Temporary file-directory for saving dataset: '{temp_dir}'\n")

In [ ]:
# for downloading from google-drive, use method described
# elsewhere (https://stackoverflow.com/a/39225272)
# with some modifications

def download_file_from_google_drive(URL, destination_dir):
    # Funktion zum Herunterladen einer Datei von Google Drive
    session = requests.Session()

    response = session.get(URL, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {"confirm": token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination_dir)


def get_confirm_token(response):
    # Funktion zum Abrufen des Bestätigungstokens für den Download
    for key, value in response.cookies.items():
        if key.startswith("download_warning"):
            return value

    return None


def save_response_content(response, destination_dir):
    # Funktion zum Speichern des Dateiinhaltes
    CHUNK_SIZE = 32768

    with open(destination_dir + "/data.zip", "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

# Die Datei von Google Drive herunterladen
download_file_from_google_drive(ds_url, temp_dir)

print("Folder structure inside 'tempdir':\n")
print(os.listdir(temp_dir))

In [ ]:
# Entpacken der heruntergeladenen Zip-Datei
shutil.unpack_archive(filename=temp_dir + "/data.zip", extract_dir=temp_dir)
print("Folder structure inside 'tempdir':\n")
print(os.listdir(temp_dir))

## Einlesen und Präprozessierung der Daten

In [ ]:
# Pfade zu den Bilderverzeichnissen festlegen und Anzahl der Bilder ausgeben
main_path = temp_dir + "/COVID-19_Radiography_Dataset"
print(os.listdir(main_path))

covid_dir = os.path.join(main_path, "COVID/images")
normal_dir = os.path.join(main_path, "Normal/images")

print("Anzahl Bilder mit COVID:", len(os.listdir(covid_dir)))
print("Anzahl normaler Bilder:", len(os.listdir(normal_dir)))

In [ ]:
# Beispielbild laden und anzeigen
example_image = cv2.imread(os.path.join(covid_dir, "COVID-1.png"))

plt.imshow(example_image)

In [ ]:
# Form des Beispielbildes ausgeben
print(example_image.shape)

In [ ]:
# Funktion zum Laden und Vorverarbeiten von Bildern aus einem Verzeichnis
def loadImages(dir, size, label):
  images = []
  labels = []

  for i in range(len(size)):
    img_path = dir + "/" + size[i]
    img = cv2.imread(img_path)
    img = img / 255.0
    img = cv2.resize(img, (100, 100))
    images.append(img)
    labels.append(label)

  images = np.asarray(images)

  return images, labels

In [ ]:
# Laden und Beschriften von COVID-Bildern (10% des Datensatzes)
# (kann bei Bedarf / je nach Verfügbarkeit der Ressourcen angepasst werden)
covid_images, covid_labels = loadImages(covid_dir, os.listdir(covid_dir)[:int(0.1*len(os.listdir(covid_dir)))], 1)
print("Covid cases:\n")
print(len(covid_images), len(covid_labels))

In [ ]:
# Ausgabe der ersten 10 Covid-Labels
print(covid_labels[0:10])

In [ ]:
# Laden und Beschriften von normalen Bildern (10% des Datensatzes)
# (kann bei Bedarf / je nach Verfügbarkeit der Ressourcen angepasst werden)
normal_images, normal_labels = loadImages(normal_dir, os.listdir(normal_dir)[:int(0.1*len(os.listdir(normal_dir)))], 0)
print("Normal cases:\n")
print(len(normal_images), len(normal_labels))

In [ ]:
# Ausgabe der ersten 10 Labels von normalen Fällen
print(normal_labels[0:10])

## Aufteilung der Daten

In [ ]:
# Kombinieren der COVID- und normalen Bilddaten und Labels
x = np.r_[covid_images, normal_images]

y = np.r_[covid_labels, normal_labels]

In [ ]:
# Aufteilen der Daten in Trainings- und Testsets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15)

## CNN-Architektur

In [ ]:
# Form des ersten Testbildes ausgeben
print(x_test[0].shape)

In [ ]:
# Erstellen und Zusammenfassen der CNN-Modellarchitektur
model = Sequential()

model.add(Conv2D(32, 3, input_shape = (100, 100, 3), activation = "relu"))
model.add(MaxPooling2D())

model.add(Conv2D(16, 3, activation = "relu"))
model.add(MaxPooling2D())

model.add(Conv2D(16, 3, activation = "relu"))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

model.summary()

## Modelltraining und -evaluation

In [ ]:
# Kompilieren des Modells mit Optimizer, Verlustfunktion und Metriken
model.compile(optimizer = "adam",
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

In [ ]:
# Trainieren des Modells (für 5 Epochen; kann bei Bedarf /
# je nach Verfügbarkeit von Ressourcen angepasst werden)
model.fit(x_train, y_train,
          batch_size = 32, epochs = 5,
          validation_data = (x_test, y_test))

In [ ]:
# Plotten der Trainings- und Testgenauigkeit
plt.plot(model.history.history["accuracy"], label = "train_accuracy")
plt.plot(model.history.history["val_accuracy"], label = "test_accuracy")
plt.legend()
plt.show()

In [ ]:
# Evaluieren des Modells auf den Testdaten
eval_results = model.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_results)

In [ ]:
# Vorhersagen mit dem Modell treffen und in binäre Klassen umwandeln
y_pred = model.predict(x_test)

y_pred_bin = (y_pred > 0.5).astype(int)

In [ ]:
# Ausgabe von Beispielbildern inklusive vorhergesagtem Label

def prediction_to_class(prediction: int):
  return "Covid" if prediction == 0 else "normal"


for i in range(len(x_test)):
  plt.figure()
  plt.imshow(x_test[i])
  plt.title(f"Ground-Truth: {prediction_to_class(y_test[i])}, Prediction: {prediction_to_class(y_pred_bin[i])}")
  plt.show()